<a href="https://colab.research.google.com/github/SungkooLee/neowizard/blob/master/PPTX_to_PDF_%EB%B3%80%ED%99%98%EA%B8%B0_(Gemini_%EC%97%B0%EB%8F%99).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
import os
import sys
import threading
import time
import re

# GUI 라이브러리
import tkinter as tk
from tkinter import ttk, filedialog, messagebox

# 기능별 라이브러리
import comtypes.client
from pptx import Presentation
import fitz  # PyMuPDF
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import google.generativeai as genai

# --- 전역 설정 ---
APP_TITLE = "PPTX to PDF 변환기 (Gemini 연동)"
APP_GEOMETRY = "800x750"

# --- 핵심 기능 클래스 ---
class PptxConverter:
    """PPTX 변환 및 Gemini 연동 로직을 처리하는 클래스"""

    def __init__(self, api_key, status_callback, progress_callback):
        self.api_key = api_key
        self.status_callback = status_callback
        self.progress_callback = progress_callback
        self.gemini_model = None
        if self.api_key:
            try:
                genai.configure(api_key=self.api_key)
                self.gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')
            except Exception as e:
                self.log_error(f"Gemini API 키 설정 실패: {e}")

    def log_status(self, message):
        """UI에 상태 메시지를 업데이트"""
        if self.status_callback:
            self.status_callback(message)

    def log_error(self, message):
        """UI에 에러 메시지를 표시"""
        messagebox.showerror("오류", message)

    def update_progress(self, value):
        """UI의 진행률 바를 업데이트"""
        if self.progress_callback:
            self.progress_callback(value)

    def extract_text_from_pptx(self, pptx_path):
        """python-pptx를 사용하여 PPTX 파일에서 텍스트를 추출"""
        try:
            self.log_status(f"'{os.path.basename(pptx_path)}'에서 텍스트 추출 중...")
            prs = Presentation(pptx_path)
            full_text = []
            for slide in prs.slides:
                slide_text = []
                for shape in slide.shapes:
                    if not shape.has_text_frame:
                        continue
                    for paragraph in shape.text_frame.paragraphs:
                        for run in paragraph.runs:
                            slide_text.append(run.text)
                full_text.append(' '.join(slide_text))
            return '\n'.join(full_text)
        except Exception as e:
            self.log_error(f"PPTX 텍스트 추출 실패: {e}")
            return None

    def convert_to_pdf_native(self, pptx_path, output_folder):
        """PowerPoint COM 객체를 사용하여 PPTX를 PDF로 변환 (고품질)"""
        self.log_status(f"'{os.path.basename(pptx_path)}'를 PDF로 변환 중 (PowerPoint 사용)...")
        powerpoint = None
        output_path = None
        try:
            powerpoint = comtypes.client.CreateObject("Powerpoint.Application")
            deck = powerpoint.Presentations.Open(pptx_path, ReadOnly=True, WithWindow=False)

            filename = os.path.splitext(os.path.basename(pptx_path))[0]
            output_path = os.path.join(output_folder, f"{filename}.pdf")

            deck.SaveAs(output_path, 32)  # 32 is the ppSaveAsPDF format code
            deck.Close()
            self.log_status(f"PDF 변환 완료: {os.path.basename(output_path)}")
            return output_path
        except Exception as e:
            self.log_error(f"PowerPoint를 이용한 PDF 변환 실패: {e}\nPowerPoint가 설치되어 있는지 확인하세요.")
            return None
        finally:
            if powerpoint:
                powerpoint.Quit()

    def translate_text(self, text, target_language):
        """Gemini를 사용하여 텍스트 번역"""
        if not self.gemini_model:
            self.log_error("Gemini 모델이 초기화되지 않았습니다. API 키를 확인하세요.")
            return None

        self.log_status(f"{target_language}(으)로 번역 요청 중...")
        try:
            prompt = f"Translate the following presentation text into {target_language}. Maintain the structure and line breaks as much as possible:\n\n---\n\n{text}"
            response = self.gemini_model.generate_content(prompt)
            self.log_status("번역 완료.")
            return response.text
        except Exception as e:
            self.log_error(f"Gemini 번역 실패: {e}")
            return None

    def create_pdf_from_text(self, text, output_path):
        """ReportLab을 사용하여 텍스트로부터 PDF 생성"""
        self.log_status(f"번역된 텍스트로 PDF 생성 중: {os.path.basename(output_path)}")
        try:
            # 윈도우에 내장된 한국어 지원 폰트 경로
            font_path = "c:/Windows/Fonts/malgun.ttf"
            if not os.path.exists(font_path):
                # 대체 폰트 (돋움)
                font_path = "c:/Windows/Fonts/dotum.ttf"

            pdfmetrics.registerFont(TTFont('MalgunGothic', font_path))

            c = canvas.Canvas(output_path, pagesize=letter)
            width, height = letter

            text_object = c.beginText(40, height - 40)
            text_object.setFont('MalgunGothic', 10)

            for line in text.split('\n'):
                text_object.textLine(line)

            c.drawText(text_object)
            c.save()
            self.log_status("텍스트 PDF 생성 완료.")
        except Exception as e:
            self.log_error(f"텍스트 기반 PDF 생성 실패: {e}\n'Malgun Gothic' 폰트가 필요합니다.")

    def verify_content_with_gemini(self, original_text, pdf_path):
        """Gemini를 사용하여 원본 텍스트와 PDF 텍스트의 유사도 검증"""
        if not self.gemini_model:
            self.log_error("Gemini 모델이 초기화되지 않았습니다.")
            return "검증 불가", "API 키를 확인하세요."

        self.log_status(f"'{os.path.basename(pdf_path)}' 텍스트 추출 및 검증 시작...")

        # 1. PDF에서 텍스트 추출 (PyMuPDF)
        pdf_text = ""
        try:
            with fitz.open(pdf_path) as doc:
                for page in doc:
                    pdf_text += page.get_text()
        except Exception as e:
            self.log_error(f"PDF에서 텍스트 추출 실패: {e}")
            return "검증 실패", f"PDF 텍스트 추출 오류: {e}"

        # 2. Gemini에 검증 요청
        self.log_status("Gemini에 내용 검증 및 유사도 분석 요청 중...")
        try:
            prompt = f"""
            Analyze the semantic similarity between the following two texts.
            Text A is from the original PowerPoint file.
            Text B is from the converted PDF file.

            1.  Provide a semantic similarity score as a percentage (e.g., "Similarity: 98%"). The percentage should represent how well the core meaning, key information, and nuances of Text A are preserved in Text B.
            2.  Provide a brief, one-sentence summary of your verification result.

            --- TEXT A (Original) ---
            {original_text}

            --- TEXT B (Converted) ---
            {pdf_text}
            """
            response = self.gemini_model.generate_content(prompt)

            # 3. 결과 파싱
            similarity_match = re.search(r"(\d{1,3})\s*%", response.text)
            similarity_percent = similarity_match.group(1) if similarity_match else "N/A"

            self.log_status("내용 검증 완료.")
            return f"{similarity_percent}%", response.text

        except Exception as e:
            self.log_error(f"Gemini 내용 검증 실패: {e}")
            return "검증 실패", str(e)


# --- GUI 애플리케이션 클래스 ---
class App(tk.Tk):
    """메인 GUI 애플리케이션 클래스"""
    def __init__(self):
        super().__init__()

        self.title(APP_TITLE)
        self.geometry(APP_GEOMETRY)

        # 스타일 설정
        style = ttk.Style(self)
        style.theme_use('vista') # 'clam', 'alt', 'default', 'classic', 'vista', 'xpnative'

        # 변수 초기화
        self.input_files = []
        self.output_folder = tk.StringVar(value=os.path.join(os.path.expanduser("~"), "Desktop"))
        self.gemini_api_key = tk.StringVar()
        self.translate_var = tk.BooleanVar(value=False)
        self.target_language = tk.StringVar(value="English")

        # 메인 프레임
        main_frame = ttk.Frame(self, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)

        self._create_widgets(main_frame)

    def _create_widgets(self, parent):
        # 1. API 키 설정 프레임
        api_frame = ttk.LabelFrame(parent, text="1. Gemini API 설정", padding="10")
        api_frame.pack(fill=tk.X, padx=5, pady=5)
        ttk.Label(api_frame, text="API Key:").pack(side=tk.LEFT, padx=(0, 5))
        ttk.Entry(api_frame, textvariable=self.gemini_api_key, width=60, show="*").pack(side=tk.LEFT, fill=tk.X, expand=True)

        # 2. 파일 선택 프레임
        files_frame = ttk.LabelFrame(parent, text="2. 원본 PPTX 파일 선택", padding="10")
        files_frame.pack(fill=tk.X, padx=5, pady=5)

        self.files_listbox = tk.Listbox(files_frame, height=8, selectmode=tk.EXTENDED)
        self.files_listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 5))

        files_buttons_frame = ttk.Frame(files_frame)
        files_buttons_frame.pack(side=tk.LEFT, fill=tk.Y)
        ttk.Button(files_buttons_frame, text="파일 추가", command=self.select_files).pack(fill=tk.X, pady=2)
        ttk.Button(files_buttons_frame, text="목록 지우기", command=self.clear_files).pack(fill=tk.X, pady=2)

        # 3. 저장 폴더 프레임
        output_frame = ttk.LabelFrame(parent, text="3. 저장할 폴더 선택", padding="10")
        output_frame.pack(fill=tk.X, padx=5, pady=5)
        ttk.Entry(output_frame, textvariable=self.output_folder, state="readonly", width=70).pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 5))
        ttk.Button(output_frame, text="폴더 선택", command=self.select_output_folder).pack(side=tk.LEFT)

        # 4. 변환 옵션 프레임
        options_frame = ttk.LabelFrame(parent, text="4. 변환 옵션", padding="10")
        options_frame.pack(fill=tk.X, padx=5, pady=5)

        ttk.Checkbutton(options_frame, text="Gemini로 번역 후 PDF로 저장 (선택된 파일들을 번역하여 새 PDF 생성)", variable=self.translate_var).pack(anchor=tk.W)

        lang_frame = ttk.Frame(options_frame)
        lang_frame.pack(fill=tk.X, padx=20, pady=5)
        ttk.Label(lang_frame, text="번역할 언어:").pack(side=tk.LEFT, padx=(0, 5))
        ttk.Combobox(lang_frame, textvariable=self.target_language, values=["English", "Japanese", "Chinese", "Spanish", "French"], state="readonly").pack(side=tk.LEFT)

        # 5. 실행 및 진행상황 프레임
        action_frame = ttk.Frame(parent, padding="10")
        action_frame.pack(fill=tk.BOTH, expand=True)

        self.start_button = ttk.Button(action_frame, text="변환 시작", command=self.start_conversion_thread, style='Accent.TButton')
        self.start_button.pack(pady=10)
        style = ttk.Style()
        style.configure('Accent.TButton', font=('Helvetica', 12, 'bold'))

        self.progress_bar = ttk.Progressbar(action_frame, orient="horizontal", length=300, mode="determinate")
        self.progress_bar.pack(pady=5, fill=tk.X)

        self.status_label = ttk.Label(action_frame, text="대기 중...", anchor=tk.W)
        self.status_label.pack(fill=tk.X, pady=5)

    def select_files(self):
        files = filedialog.askopenfilenames(
            title="PPTX 파일을 선택하세요",
            filetypes=(("PowerPoint files", "*.pptx *.ppt"), ("All files", "*.*"))
        )
        for f in files:
            if f not in self.input_files:
                self.input_files.append(f)
                self.files_listbox.insert(tk.END, os.path.basename(f))
        self.update_status(f"{len(self.input_files)}개 파일 선택됨")

    def clear_files(self):
        self.input_files = []
        self.files_listbox.delete(0, tk.END)
        self.update_status("파일 목록이 초기화되었습니다.")

    def select_output_folder(self):
        folder = filedialog.askdirectory(title="저장할 폴더를 선택하세요")
        if folder:
            self.output_folder.set(folder)
            self.update_status(f"저장 폴더가 '{folder}'로 지정되었습니다.")

    def update_status(self, message):
        self.status_label.config(text=f"상태: {message}")
        self.update_idletasks()

    def update_progress(self, value):
        self.progress_bar['value'] = value
        self.update_idletasks()

    def start_conversion_thread(self):
        # 유효성 검사
        if not self.input_files:
            messagebox.showwarning("입력 오류", "변환할 파일을 먼저 추가해주세요.")
            return
        if not self.output_folder.get():
            messagebox.showwarning("입력 오류", "저장할 폴더를 선택해주세요.")
            return
        if self.translate_var.get() and not self.gemini_api_key.get():
            messagebox.showwarning("API 키 오류", "번역 기능을 사용하려면 Gemini API 키를 입력해야 합니다.")
            return

        # 스레드 시작
        self.start_button.config(state=tk.DISABLED)
        thread = threading.Thread(target=self.run_conversion_logic, daemon=True)
        thread.start()

    def run_conversion_logic(self):
        converter = PptxConverter(self.gemini_api_key.get(), self.update_status, self.update_progress)
        total_files = len(self.input_files)
        results = []

        for i, file_path in enumerate(self.input_files):
            self.update_progress((i / total_files) * 100)

            if self.translate_var.get():
                # 번역 모드
                original_text = converter.extract_text_from_pptx(file_path)
                if original_text:
                    translated_text = converter.translate_text(original_text, self.target_language.get())
                    if translated_text:
                        filename = os.path.splitext(os.path.basename(file_path))[0]
                        output_path = os.path.join(self.output_folder.get(), f"{filename}_translated_{self.target_language.get()}.pdf")
                        converter.create_pdf_from_text(translated_text, output_path)
            else:
                # 일반 변환 및 검증 모드
                pdf_path = converter.convert_to_pdf_native(file_path, self.output_folder.get())
                if pdf_path:
                    original_text = converter.extract_text_from_pptx(file_path)
                    if original_text and self.gemini_api_key.get():
                        similarity, summary = converter.verify_content_with_gemini(original_text, pdf_path)
                        results.append(f"파일: {os.path.basename(file_path)}\n- 유사도: {similarity}\n- 검증 요약: {summary}\n")
                    elif not self.gemini_api_key.get():
                        results.append(f"파일: {os.path.basename(file_path)}\n- PDF 변환 완료 (API 키가 없어 검증은 건너뜀)\n")
                    else:
                        results.append(f"파일: {os.path.basename(file_path)}\n- PDF 변환 완료 (원본 텍스트 추출 실패로 검증 불가)\n")

        self.update_progress(100)
        self.update_status("모든 작업 완료!")

        # 완료 메시지 표시
        final_message = "모든 변환 작업이 완료되었습니다.\n\n"
        if results:
            final_message += "--- 내용 검증 결과 ---\n" + "\n".join(results)
        elif self.translate_var.get():
            final_message += "번역된 파일들이 지정된 폴더에 저장되었습니다."

        messagebox.showinfo("작업 완료", final_message)
        self.start_button.config(state=tk.NORMAL)


if __name__ == '__main__':
    # COM 객체 사용을 위한 초기화 (PyInstaller 실행 파일 생성 시 필요할 수 있음)
    if sys.platform == "win32":
        comtypes.CoInitialize()
    try:
        app = App()
        app.mainloop()
    finally:
        if sys.platform == "win32":
            comtypes.CoUninitialize()

ModuleNotFoundError: No module named 'comtypes'